# LightRay Analysys

In this notebook we will study light reflaction ans refraction, using LightRay analysis and the Snell and Fresnel equations.

In [1]:
!pip install numpy
!pip install ipympl
!pip install matplotlib

In [2]:
from math import *
import numpy as np
import matplotlib.pyplot as plt

if 'google.colab' in str(get_ipython()):
  %matplotlib inline
else:
  %matplotlib widget

## Create a the a LightRay graph class

In [3]:
class LightRay:
    """Defines a LightRay vecto"""
    def __init__(self, r_type:int, angle_deg:int, polarization:str='p', ratio:int=1): 
        """ Intialize a LightRay vector

            Args:
                type: int
                    Ray r_type -> 0 = Inciddent, 1 = Reflected and 2 = Refracted
                angle: float
                    Angle from the normal in degrees
            
            Returns: LightRay object
        """
        self.type = r_type
        self.angle = angle_deg
        self.polarization = polarization
        self.ratio = ratio


class Material:
    """Material with properties related to light interaction"""
    
    def __init__(self, name:str,  refractive_index:str, thickness_mm:float):
        """ Intialize a Material

            Args:
                name: str
                refractive_index: float
                thickness_mm: float
            
            Returns: Material object
        """
        self.name = name
        self.refractive_index = refractive_index
        self.thickness = thickness_mm


class RayGraph:
    ''' LightRay graph for multiple interfaces'''

    # def __init__(self):
    #     self.figure, self.axis = plt.subplots()
    
    def __init__(self, hight: int, width: int):
        self.figure, self.axis = plt.subplots()
        self.axis.set_ylim(-hight//2, hight//2)
        self.axis.set_xlim(-width//2, width//2)
        self.axis.set_ylabel('hight (mm)')
        self.axis.set_xlabel('width (mm)')


        interface = self.axis.plot([-width//2, width//2], [0, 0], 
                                    color='gray', linestyle='dashed', alpha= 0.5, label= 'interface')
        normal = self.axis.plot([0, 0], [-width//2, width//2], 
                                color='yellow', linestyle='dashed', alpha= 0.5, label='normal')

        plt.legend()

        self.ray_color='blue'
        self.ray_alpha=0.1

    def plot_vector(self, vector: list = [(0, 0),(0, 0)]):
        """ Plot a vector based on the start and end point
            
            Args: 
                vector: list[(start_x, start_y), (end_x, end_y)]
            """
        a = vector[0]
        b = vector[1]
        dx = b[0] - a[0]
        dy = b[1] - a[1]

        magnitude = sqrt(dx**2+dy**2)
        # head_length = magnitude * 0.05
        head_length = 0.2

        dx = dx / magnitude
        dy = dy / magnitude

        magnitude = magnitude - head_length

        self.axis.arrow(a[0], a[1], magnitude*dx, magnitude*dy, 
                        head_width=head_length, head_length=head_length, 
                        color=self.ray_color,alpha=self.ray_alpha)
    
    def plot_snell_rays(self, incident_ray: LightRay, material_1: Material, material_2: Material):
        """Plot the refracted ray for a 2 material interface based on the snell law"""
        incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
        incident_y_start = material_1.thickness
        incident_x_end = 0
        incident_y_end = 0

        self.plot_vector([(incident_x_start,incident_y_start),
                          (incident_x_end,incident_y_end)])

        resultant_ray = snell_resultant(incident_ray, material_1, material_2)

        resultant_x_start = 0
        resultant_y_start = 0

        if resultant_ray.type == 1: # Total internal reflection ray
            resultant_x_end = tan(radians(resultant_ray.angle))*material_1.thickness
            resultant_y_end = material_1.thickness
        elif resultant_ray.type == 2: # Refracted ray
            resultant_x_end = tan(radians(resultant_ray.angle))*material_2.thickness
            resultant_y_end = -material_2.thickness
    
        self.plot_vector([(resultant_x_start,resultant_y_start),
                        (resultant_x_end,resultant_y_end)])
    
    def plot_fresnel_rays(self, incident_ray: LightRay, material_1: Material, material_2: Material):
        """Plot the refracted ray for a 2 material interface based on the snell law"""
        incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
        incident_y_start = material_1.thickness
        incident_x_end = 0
        incident_y_end = 0
        
        self.ray_alpha = 1
        self.plot_vector([(incident_x_start,incident_y_start),
                          (incident_x_end,incident_y_end)])

        reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)

        reflected_x_start = 0
        reflected_y_start = 0
        reflected_x_end = tan(radians(reflected_ray.angle))*material_1.thickness
        reflected_y_end = material_1.thickness
        
        self.ray_alpha = reflected_ray.ratio
        self.plot_vector([(reflected_x_start,reflected_y_start),
                        (reflected_x_end,reflected_y_end)])

        transmitted_x_start = 0
        transmitted_y_start = 0
        transmitted_x_end = tan(radians(reflected_ray.angle))*material_2.thickness
        transmitted_y_end = -material_2.thickness

        self.ray_alpha = transmitted_ray.ratio
        self.plot_vector([(transmitted_x_start,transmitted_y_start),
                        (transmitted_x_end,transmitted_y_end)])

    def show(self):
        plt.show()

## Implement Snell's and Fresnel's equations


In [4]:
def snell_resultant(incident_ray: LightRay, material_1: Material, material_2: Material) -> LightRay:
    """ Compute the refracted ray using the snells law"""

    n1 = material_1.refractive_index # Refractive index of the first materia
    n2 = material_2.refractive_index # Refractive index of the second material

    argument = (n1/n2)*sin(radians(incident_ray.angle))

    if argument <=  1:
        refraction_ang = asin(argument)
        resultant_ray = LightRay(2, degrees(refraction_ang))
    else:
        resultant_ray = LightRay(1, incident_ray.angle)

    return resultant_ray

    
def fresnel_resultant(incident_ray: LightRay, material_1: Material, material_2: Material) -> LightRay:
    """ Compute the refracted ray using the snells law"""
    n1 = material_1.refractive_index # Refractive index of the first materia
    n2 = material_2.refractive_index # Refractive index of the second material
    
    polarization = incident_ray.polarization
    incicent_angle = radians(incident_ray.angle)

    # using snell's law to get the transmited angle
    snell_argument = n1*sin(radians(incident_ray.angle))/n2

    # verify the total internal reflaction condition
    if snell_argument <=  1: 
        refraction_ang = asin(snell_argument)
        if polarization == 's':
            reflectance = ((n1*cos(incicent_angle)-n2*cos(refraction_ang))/(n1*cos(incicent_angle)+n2*cos(refraction_ang)))**2
        elif polarization == 'p':
            reflectance = ((n1*cos(refraction_ang)-n2*cos(incicent_angle))/(n1*cos(refraction_ang)+n2*cos(incicent_angle)))**2
        reflected_ray = LightRay(1, degrees(refraction_ang),'p',reflectance)
        transmitted_ray = LightRay(2, degrees(refraction_ang),'p',1-reflectance)
    else:
        refraction_ang = incicent_angle
        reflected_ray = LightRay(1, degrees(refraction_ang),'p',1)
        transmitted_ray = LightRay(2, degrees(refraction_ang),'p',0)
    
    return reflected_ray, transmitted_ray

## Define a lightray experiment 

Plot the resultant rays for different angles and polarizations based on the snell's and fresnel's laws

In [5]:
def lightray_experiment(mode:str, incident_ray: LightRay, material_1: Material, material_2: Material) -> None:
    angles = np.linspace(0.001, 90, num=10)

    lightray_graph = RayGraph(15,15)
    lightray_graph.ray_color = [1.0, 0.0, 0.0, 1.0]
    lightray_graph.ray_alpha = 1.0

    if mode == 'snell':
        lightray_graph.axis.set_title('Snell analysis - {}/{}'.format(material_1.name,
                                                                        material_2.name))
    elif mode == 'fresnel':
        lightray_graph.axis.set_title('Fresnel analysis - {}/{} \n incident ray polarization = {}'.format(material_1.name,
                                                                                                    material_2.name, 
                                                                                                    incident_ray.polarization))

    for angle in angles:
        if lightray_graph.ray_color[2] + 1/len(angles) < 1:
            lightray_graph.ray_color[2] += 1/len(angles) 
            lightray_graph.ray_color[0] -= 1/len(angles) 
            incident_ray.angle = angle
        if mode == 'snell':
            lightray_graph.plot_snell_rays(incident_ray,material_1,material_2)
        elif mode == 'fresnel':
            lightray_graph.plot_fresnel_rays(incident_ray,material_1,material_2)

    lightray_graph.show()

def reflectance_experiment(material_1: Material, material_2: Material) -> None:
    angles = np.linspace(0.001, 90, num=100)

    fig, ax = plt.subplots()
    ax.set_title('Fresnell reflectance analysis - {}/{}'.format(material_1.name,
                                                                material_2.name))

    ax.set_xlabel("angle (dgrees)")
    ax.set_xlim(0, 90)
    p_reflectance = []
    p_transmittance = []
    s_reflectance = []
    s_transmittance = []

    incident_ray = LightRay(0, 0.001, polarization='p'
    )
    for angle in angles:
        incident_ray.angle = angle
        incident_ray.polarization = 'p'
        reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)
        p_reflectance.append(reflected_ray.ratio)
        p_transmittance.append(1-reflected_ray.ratio)

        incident_ray.polarization = 's'
        reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)
        s_reflectance.append(reflected_ray.ratio)
        s_transmittance.append(1-reflected_ray.ratio)
        
    ax.plot(angles, p_reflectance, 'b', label='Rp')
    ax.plot(angles, p_transmittance, 'r', label='Tp')
    ax.plot(angles, s_reflectance, 'b--', label='Rs')
    ax.plot(angles, s_transmittance, 'r--', label='Ts')
    plt.legend()
    plt.show()

### Plot the refracted ray using the snell's and fresnel's law for vacuum and water for different angles and polarizations

In [6]:
plt.close('all')

material_1 = Material('vacuum', 1.0, 5.0)
material_2 = Material('water', 1.33, 5.0)

reflectance_experiment(material_1, material_2)


incident_ray = LightRay(0, 0.001, polarization='p')
lightray_experiment('snell',incident_ray, material_1, material_2)
lightray_experiment('fresnel',incident_ray, material_1, material_2)

incident_ray = LightRay(0, 0.001, polarization='s')
lightray_experiment('fresnel',incident_ray, material_1, material_2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plot the refracted ray using the snell's and fresnel's law for optoc fiber and fbg for different angles and polarizations

In [7]:
material_1 = Material('optic_fiber', 1.45, 5)
material_2 = Material('fbg', 1.4005, 5.0)

reflectance_experiment(material_1, material_2)

incident_ray = LightRay(0, 0.001, polarization='p')
lightray_experiment('snell',incident_ray, material_1, material_2)
lightray_experiment('fresnel',incident_ray, material_1, material_2)

incident_ray = LightRay(0, 0.001, polarization='s')
lightray_experiment('fresnel',incident_ray, material_1, material_2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …